<h2 align='center'> Stainless Steel Classification </h3>

<p> The General idea of this notebook is to highlight all the process and subprocesses done for Stainless Steel Classification.<br>For this project we have used the dataset provided <a href="https://www.kaggle.com/code/kerneler/starter-tig-stainless-steel-304-ee6df282-0/data">Here</a>. </p>

<h4> <u> Dataset Discussion </u> </h4>
<div>
    <img src='Assets\Dataset_Labels.png' align='right' width=600 height=300>
    <p align='left'><br>Our Dataset has been label encoded as following: <br> The idea is that provided an image our classifier will be able to classify the results as one of the following labels. <br><br>["good weld","burn through","contamination","lack of fusion","lack of shielding gas","high travel speed"]<br><br>
    Since we are following supervised approach we have a json file which maps each image with corresponding output labels which can be used to map each image with corresponding labels. <br><br> The mentioned json file can be seen below: <br> <br></p>
    <img src='Assets\json_files.png' width=550>
</div>


<h3> 0. Intalling Dependencies </h3>

In [ ]:
!pip install tensorflow 
!pip install -U scikit-learn
!pip install numpy
!pip install pillow


<h3> 1. Importing Dependencies </h3>

Let's import all the dependencies so we can use it later

In [2]:
import json
import numpy as np 
from PIL import Image
import tensorflow as tf


np.set_printoptions(threshold=sys.maxsize)

<h3> 1. PreProcessing </h3>

<p> Once Downloaded please unzip the dataset on the same working directory and the folder directory should be: <br>

<i>../ss304/train/train.json</i>
</p>


In [3]:
def convert_to_array(i,features,f):

    if f==0:
        image=features[i]
        
    else: 
        image=i
    
    image = Image.open(image)
    image=image.resize((200,200))
    numpydata = np.asarray(image)
    
    return numpydata


def get_images_array_and_labels(path):
    print('Getting all the images, Resizing it and converting it to numpy array so that it can be fitted on CNN. It might take some time depending upon processing speed')
    with open(path,'r') as datasets:
        datasets=json.load(datasets)
    features=list(datasets.keys())
    features=['/'.join(path.split('\\')[0:-1])+'/'+str(i) for i in features]
    features=np.array([convert_to_array(i,features,0) for i in range(len(features))])
    labels=np.array(list(datasets.values())).reshape(-1,1)
    #labels=tf.keras.utils.to_categorical(labels)
    return features,labels
    
    

X,Y=get_images_array_and_labels('ss304\\train\\train.json')
Xtest,Ytest=get_images_array_and_labels('ss304\\test\\test.json')

Getting all the images, Resizing it and converting it to numpy array so that it can be fitted on CNN. It might take some time depending upon processing speed


KeyboardInterrupt: 

In [4]:
def create_cnn_model():
    model=tf.keras.models.Sequential()

    #Convlution Process Starts here.
    model.add(tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(200, 200, 1)))
    model.add(tf.keras.layers.MaxPool2D((2,2)))
    
    model.add(tf.keras.layers.Conv2D(32,(3,3),activation='relu'))
    model.add(tf.keras.layers.MaxPool2D((2,2)))
    
    #model.add(tf.keras.layers.Conv2D(64,(3,3),activation='relu'))
    #model.add(tf.keras.layers.MaxPool2D((2,2)))
    
    #model.add(tf.keras.layers.Conv2D(128,(3,3),activation='relu'))
    #model.add(tf.keras.layers.MaxPool2D((2,2)))
    
    #model.add(tf.keras.layers.Conv2D(128,(3,3),activation='relu'))
    #model.add(tf.keras.layers.MaxPool2D((2,2)))
    
    #model.add(tf.keras.layers.Conv2D(64,(3,3),activation='relu'))
    #model.add(tf.keras.layers.MaxPool2D((2,2)))
    
    #Fully Connected Layer from here 
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(32,activation='relu',kernel_regularizer=tf.keras.regularizers.L2(l2=0.001)))
    model.add(tf.keras.layers.Dense(16,activation='relu',kernel_regularizer=tf.keras.regularizers.L2(l2=0.001)))
    model.add(tf.keras.layers.Dense(6,activation='softmax',kernel_regularizer=tf.keras.regularizers.L2(l2=0.001))) 
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    
    #Compile the CNN Model
    model.compile(optimizer=opt,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
    model.summary()
    return model
    
CNN_model=create_cnn_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 16)      160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 99, 99, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 48, 48, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 73728)             0         
                                                                 
 dense (Dense)               (None, 32)                2

In [14]:
#print(X_test[1].shape)
history_for_plotting=CNN_model.fit(X,Y,epochs=20,batch_size=128,validation_data=(Xtest,Ytest),shuffle=True)

Epoch 1/20
190/190 [==============================] - 100s 526ms/step - loss: 2.8983 - accuracy: 0.4021 - val_loss: 1.4452 - val_accuracy: 0.4173
Epoch 2/20
190/190 [==============================] - 100s 527ms/step - loss: 0.7689 - accuracy: 0.5602 - val_loss: 1.2451 - val_accuracy: 0.7720
Epoch 3/20
190/190 [==============================] - 104s 547ms/step - loss: 0.4801 - accuracy: 0.8442 - val_loss: 1.0323 - val_accuracy: 0.8217
Epoch 4/20
190/190 [==============================] - 103s 541ms/step - loss: 0.2475 - accuracy: 0.9096 - val_loss: 0.7822 - val_accuracy: 0.8172
Epoch 5/20
190/190 [==============================] - 103s 540ms/step - loss: 0.1996 - accuracy: 0.9456 - val_loss: 1.9657 - val_accuracy: 0.8707
Epoch 6/20
190/190 [==============================] - 102s 535ms/step - loss: 0.1557 - accuracy: 0.9963 - val_loss: 1.3682 - val_accuracy: 0.8651
Epoch 7/20
190/190 [==============================] - 104s 547ms/step - loss: 0.1269 - accuracy: 0.9989 - val_loss: 0.6995 -

In [15]:
CNN_model.save('Stainless_Steel_Classification_model.h5')

<h3> [Last Step No] Inference Testing </h3>

In [7]:
#var1='ss304/train/160705-114214-50mmLens w.s.75cm.m/image-0001.png' #op=5,high travel speed
#var2='ss304/test/160705-121434-50mmLens added slug/image-0001.png'  #op=2,contaimination
#var3='ss304/valid/160705-114925-50mmLens w.s.16cm.m/image-0150.png' #op=0,good weld 
#var4='ss304/test/160708-145105-50mmLens 350A w.s.Lev16 g.f.20L.m try joining 10mm Plate/image-0042.png' #op=3, lack of fusion


CNN_model=tf.keras.models.load_model('Stainless_Steel_Classification_model.h5')
var_input=input('Enter path of the image to test: ')
op_list=["good weld","burn through","contamination","lack of fusion","lack of shielding gas","high travel speed"]
test_image=convert_to_array(var_input,[],1)
prediction=CNN_model.predict(test_image.reshape(1,200,200,1))
print(prediction)
prediction=np.argmax(prediction)
print(prediction)
print(op_list[prediction])

AttributeError: module 'keras.api._v2.keras' has no attribute 'load_model'